In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2
# convert data to torch.FloatTensor
transform = transforms.ToTensor()
# choose the training and testing datasets
train_data = datasets.MNIST(root = 'data', train = True, download = True, transform = transform)
test_data = datasets.MNIST(root = 'data', train = False, download = True, transform = transform)
# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_index, valid_index = indices[split:], indices[:split]
# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(valid_index)

In [3]:
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, 
                                           sampler = train_sampler, num_workers = num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size,
                                          sampler = valid_sampler, num_workers = num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size,
                                         num_workers = num_workers)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
# define NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        ### self.droput = nn.Dropout(0.2)
        
    def forward(self,x):
        # flatten image input
        x = x.view(-1,28*28)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        ### x = self.droput(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        ### x = self.droput(x)
        x = F.relu(self.fc3(x))
        # add output layer
        x = self.fc4(x)
        return x
# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=10, bias=True)
)


In [5]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer 
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [9]:
# number of epochs to train the model
n_epochs = 1000
patience = 15
patience_counter = 0
# initialize tracker for minimum validation loss
valid_loss_min = np.Inf  # set initial "min" to infinity
for epoch in range(n_epochs):
    # monitor losses
    train_loss = 0
    valid_loss = 0
         
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data,label in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output,label)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item() * data.size(0)
                
    ######################    
    # validate the model #
    ######################
    model.eval()  # prep model for evaluation
    for data,label in valid_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output,label)
        # update running validation loss 
        valid_loss = loss.item() * data.size(0)
    
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tPatience Counter: {}'.format(
        epoch+1, 
        train_loss,
        valid_loss,
        patience_counter
        ))
    
    # save model if validation loss has decreased
    if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss
        patience_counter = 0
    elif patience_counter < patience:
        patience_counter += 1
    else:
        break
        

Epoch: 1 	Training Loss: 0.017004 	Validation Loss: 0.000003 	Patience Counter: 0
Validation loss decreased (inf --> 0.000003).  Saving model ...
Epoch: 2 	Training Loss: 0.012593 	Validation Loss: 0.000000 	Patience Counter: 0
Validation loss decreased (0.000003 --> 0.000000).  Saving model ...
Epoch: 3 	Training Loss: 0.015400 	Validation Loss: 0.000000 	Patience Counter: 0
Epoch: 4 	Training Loss: 0.012449 	Validation Loss: 0.000001 	Patience Counter: 1
Epoch: 5 	Training Loss: 0.011880 	Validation Loss: 0.001341 	Patience Counter: 2
Epoch: 6 	Training Loss: 0.011932 	Validation Loss: 0.000004 	Patience Counter: 3
Epoch: 7 	Training Loss: 0.010333 	Validation Loss: 0.000000 	Patience Counter: 4
Validation loss decreased (0.000000 --> 0.000000).  Saving model ...
Epoch: 8 	Training Loss: 0.011734 	Validation Loss: 0.000000 	Patience Counter: 0
Validation loss decreased (0.000000 --> 0.000000).  Saving model ...
Epoch: 9 	Training Loss: 0.011220 	Validation Loss: 0.000002 	Patience Co

In [10]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
model.eval() # prep model for evaluation
for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.192518

Test Accuracy of     0: 99% (974/980)
Test Accuracy of     1: 99% (1125/1135)
Test Accuracy of     2: 97% (1009/1032)
Test Accuracy of     3: 97% (989/1010)
Test Accuracy of     4: 97% (962/982)
Test Accuracy of     5: 95% (851/892)
Test Accuracy of     6: 97% (934/958)
Test Accuracy of     7: 97% (998/1028)
Test Accuracy of     8: 97% (949/974)
Test Accuracy of     9: 95% (963/1009)

Test Accuracy (Overall): 97% (9754/10000)
